## Prueba Final

In [ ]:
!pip install -q google-generativeai

In [2]:
import google.generativeai as genai
from google.colab import userdata

In [3]:
API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)

In [4]:
model = genai.GenerativeModel("gemini-2.5-flash")

In [5]:
class MotorIA:
  def __init__(self, model):
    self.model = model

  def generar(self, prompt:str) -> str:
    response = self.model.generate_content(prompt)
    return response.text.strip()

In [6]:
def construir_prompt(actividad: dict) -> str:
  prompt_actividad = f"""
Eres un experto en levantamiento de cargas laborales.

Analiza la siguiente actividad y entrega recomendaciones
claras y prácticas, teniendo en cuenta el cargo de la persona y la vicepresidencia en la que trabaja.

Actividad:
- Vicepresidencia: {actividad["vicepresidencia"]}
- Cargo: {actividad["cargo"]}
- Unidad de medida: {actividad["unidad_medida"]}
- Descripción: {actividad["descripcion"]}
- Frecuencia: {actividad["frecuencia"]}
- Volumen: {actividad["volumen"]}
- Tiempo real (min): {actividad["tiempo_real_min"]}
- Proceso del área: {actividad["proceso_area"]}
- Autonomía: {actividad["autonomia"]}%
- Etapa PHVA: {actividad["phva"]}
- Observaciones: {actividad["observaciones"]}

Entrega de forma resumida y concisa:
- Riesgos
- Oportunidades de mejora
- Recomendaciones concretas
"""
  return prompt_actividad

In [7]:
import time

def medir_tiempo_real():
  input("\nPresiona ENTER para INICIAR la actividad...")
  inicio = time.time()

  input("Presiona ENTER para FINALIZAR la actividad...")
  fin = time.time()

  segundos = fin-inicio
  minutos = round(segundos/60, 2)

  print(f"\nTiempo registrado: {minutos} minutos.")

  return minutos

In [8]:
def formulario_actividad():
    print("\n📝 Registro de actividad\n")

    actividad = {}

    actividad["vicepresidencia"] = input(
        "1️⃣ ¿Cuál es su Vicepresidencia?:\n> "
    )

    actividad["cargo"] = input(
        "1️⃣ ¿Cuál es su cargo?:\n> "
    )

    actividad["descripcion"] = input(
        "1️⃣ Describe la actividad que realizas:\n> "
    )

    actividad["unidad_medida"] = input(
        "2️⃣ Unidad de medida (Correo, Reunión, Documento, etc.):\n> "
    )

    actividad["frecuencia"] = input(
        "3️⃣ Frecuencia (diaria / semanal / mensual / anual):\n> "
    )

    actividad["volumen"] = int(input(
        "4️⃣ Volumen según la frecuencia:\n> "
    ))

    #actividad["tiempo_real_min"] = medir_tiempo_real()
    actividad["tiempo_real_min"] = int(input(
        "5️⃣ Tiempo real (minutos) en esta actividad:\n> "
    ))

    actividad["proceso_area"] = input(
        "6️⃣ Proceso del área al que pertenece:\n> "
    )

    actividad["autonomia"] = int(input(
        "7️⃣ Autonomía (%) en esta actividad:\n> "
    ))

    actividad["phva"] = input(
        "8️⃣ Etapa PHVA (Planear / Hacer / Verificar / Actuar):\n> "
    )

    actividad["observaciones"] = input(
        "9️⃣ Observaciones (opcional):\n> "
    )

    return actividad


In [9]:
estado = {
    "rol": None,
    "actividades": [],
    "actividad_actual": None
}

In [10]:
def seleccionar_rol():
  print("Seleccione su rol:")
  print("1-Registro de Actividades")
  print("2-Analista de Cargas")

  opcion = input("Opcion: ")

  if opcion == "1":
    return "registro"
  elif opcion == "2":
    return "analista"
  else:
    print("Opcion no valida")
    return seleccionar_rol()

In [11]:
def factor_frecuencia(frecuencia):
  frecuencia = frecuencia.lower()

  if frecuencia == "diaria":
    return 1
  if frecuencia == "semanal":
    return 1/5
  if frecuencia == "mensual":
    return 1/21
  if frecuencia == "anual":
    return 1/252
  if frecuencia == "trimestral":
    return 1/72
  if frecuencia == "quincenal":
    return 1/10
  if frecuencia == "bimestral":
    return 1/42
  if frecuencia == "semestral":
    return 1/126

  return 0

In [12]:
def calcular_metricas_actividad(actividad, total_minutos_mes=None):
    minutos_base = actividad["tiempo_real_min"]
    volumen = actividad["volumen"]
    factor = factor_frecuencia(actividad["frecuencia"])
    autonomia = actividad["autonomia"] / 100

    minutos_diarios = minutos_base * volumen * factor
    minutos_mes = minutos_diarios * 21

    minutos_w_mes = 220 * 60
    carga_w_sin_tm = minutos_mes / minutos_w_mes
    carga_trabajo_individual = carga_w_sin_tm * autonomia

    resultado = {
        "minutos_diarios": minutos_diarios,
        "minutos_mes": minutos_mes,
        "horas_diarias": minutos_diarios / 60,
        "carga_w_sin_tm": carga_w_sin_tm,
        "carga_trabajo_individual": carga_trabajo_individual
    }

    if total_minutos_mes and total_minutos_mes > 0:
        resultado["participacion_actividad_total"] = (
            minutos_mes / total_minutos_mes
        ) * 100

    return resultado

In [13]:
def calcular_resumen_global(actividades):
    carga_w_por_phva = {}
    cantidad_por_phva = {}

    carga_w_por_frecuencia = {}
    cantidad_por_frecuencia = {}

    carga_w_por_proceso_area = {}
    cantidad_por_proceso_area = {}

    # =========================
    # Acumuladores globales
    # =========================
    total_carga_w_sin_tm = 0
    total_carga_trabajo_individual = 0
    total_minutos_diarios = 0

    # =========================
    # 1. Acumulación
    # =========================
    for act in actividades:
        metricas = calcular_metricas_actividad(act)
        act["metricas"] = metricas

        phva = act.get("phva", "SIN CLASIFICAR")
        frecuencia = act.get("frecuencia", "SIN CLASIFICAR")
        proceso_area = act.get("proceso_area", "SIN CLASIFICAR")

        carga_w = metricas["carga_w_sin_tm"]
        carga_individual = metricas["carga_trabajo_individual"]
        minutos_diarios = metricas["minutos_diarios"]

        total_carga_w_sin_tm += carga_w
        total_carga_trabajo_individual += carga_individual
        total_minutos_diarios += minutos_diarios

        # ---- PHVA ----
        carga_w_por_phva.setdefault(phva, 0)
        cantidad_por_phva.setdefault(phva, 0)

        carga_w_por_phva[phva] += carga_w
        cantidad_por_phva[phva] += 1

        # ---- Frecuencia ----
        carga_w_por_frecuencia.setdefault(frecuencia, 0)
        cantidad_por_frecuencia.setdefault(frecuencia, 0)

        carga_w_por_frecuencia[frecuencia] += carga_w
        cantidad_por_frecuencia[frecuencia] += 1

        # ---- Tipo de Actividad (Proceso Área) ----
        carga_w_por_proceso_area.setdefault(proceso_area, 0)
        cantidad_por_proceso_area.setdefault(proceso_area, 0)

        carga_w_por_proceso_area[proceso_area] += carga_w
        cantidad_por_proceso_area[proceso_area] += 1

    # =========================
    # Conversión minutos
    # =========================
    minutos_diarios_empleados = total_minutos_diarios
    horas_diarias_requeridas = minutos_diarios_empleados / 60

    # =========================
    # 2. CARGA DE TRABAJO (equivale a SUMAR.SI)
    # =========================
    carga_trabajo_phva = {
        phva: round(carga, 4)
        for phva, carga in carga_w_por_phva.items()
    }

    carga_trabajo_frecuencia = {
        frecuencia: round(carga, 4)
        for frecuencia, carga in carga_w_por_frecuencia.items()
    }

    carga_trabajo_proceso_area = {
        proceso_area: round(carga, 4)
        for proceso_area, carga in carga_w_por_proceso_area.items()
    }

    suma_total_carga_phva = sum(carga_trabajo_phva.values())
    suma_total_carga_frecuencia = sum(carga_trabajo_frecuencia.values())
    suma_total_carga_proceso_area = sum(carga_trabajo_proceso_area.values())

    # =========================
    # 3. % ACTIVIDADES (SI.ERROR(C49/$C$53))
    # =========================
    porcentaje_actividades_por_phva = {
        phva: round(carga / suma_total_carga_phva, 4)
        if suma_total_carga_phva > 0 else 0
        for phva, carga in carga_trabajo_phva.items()
    }

    porcentaje_actividades_por_frecuencia = {
        frecuencia: round(carga / suma_total_carga_frecuencia, 4)
        if suma_total_carga_frecuencia > 0 else 0
        for frecuencia, carga in carga_trabajo_frecuencia.items()
    }

    porcentaje_actividades_por_proceso_area = {
        proceso_area: round(carga / suma_total_carga_proceso_area, 4)
        if suma_total_carga_proceso_area > 0 else 0
        for proceso_area, carga in carga_trabajo_proceso_area.items()
    }

    # =========================
    # Tiempo muerto
    # =========================
    jornada_laboral_diaria = 8.5

    almuerzo = 1/8
    baño_agua_etc = (3 * 5) / 60
    break_15min = 15 / 60
    latencia_software = 10 / 60
    cliente_ext_o_int = 20 / 60

    tiempo_muerto = sum([
        almuerzo / jornada_laboral_diaria,
        baño_agua_etc / jornada_laboral_diaria,
        break_15min / jornada_laboral_diaria,
        latencia_software / jornada_laboral_diaria,
        cliente_ext_o_int / jornada_laboral_diaria
    ])

    # =========================
    # RESULTADO FINAL
    # =========================
    return {
        # ---- PHVA ----
        "carga_trabajo_phva": carga_trabajo_phva,
        "total_carga_trabajo_phva": round(suma_total_carga_phva, 4),
        "porcentaje_actividades_phva": porcentaje_actividades_por_phva,
        "cantidad_actividades_phva": cantidad_por_phva,

        # ---- Frecuencia ----
        "carga_trabajo_frecuencia": carga_trabajo_frecuencia,
        "total_carga_trabajo_frecuencia": round(suma_total_carga_frecuencia, 4),
        "porcentaje_actividades_frecuencia": porcentaje_actividades_por_frecuencia,
        "cantidad_actividades_frecuencia": cantidad_por_frecuencia,

        # ---- Tipo de Actividad (Proceso Área) ----
        "carga_trabajo_proceso_area": carga_trabajo_proceso_area,
        "total_carga_trabajo_proceso_area": round(suma_total_carga_proceso_area, 4),
        "porcentaje_actividades_proceso_area": porcentaje_actividades_por_proceso_area,
        "cantidad_actividades_proceso_area": cantidad_por_proceso_area,

        # ---- Dotación ----
        "jornada_laboral_diaria": jornada_laboral_diaria,
        "factor_tiempo_neto_productivo": round((1-tiempo_muerto), 4),
        "horas_netas_efectivas_por_persona": jornada_laboral_diaria*round((1-tiempo_muerto), 4),
        "horas_diarias_requeridas": horas_diarias_requeridas,
        "numero_personas": round(horas_diarias_requeridas/(jornada_laboral_diaria*round((1-tiempo_muerto), 4)), 3),

        # ---- Totales reales ----
        "total_carga_w_sin_tm": round(total_carga_w_sin_tm, 4),
        "total_carga_trabajo_individual": round(total_carga_trabajo_individual, 4),
        "minutos_diarios_empleados": round(minutos_diarios_empleados, 2),
        "horas_diarias_requeridas": round(horas_diarias_requeridas, 2),
        "tiempo_muerto": round(tiempo_muerto, 4),
        "porcentaje_carga_trabajo_final": round(total_carga_w_sin_tm * (1 + tiempo_muerto), 4),
        "horas_diarias_requeridas_final": round(
            minutos_diarios_empleados * (1 + tiempo_muerto) / 60, 2
        )
    }

In [ ]:
motor = MotorIA(model)

actividades = []

In [ ]:
estado["rol"] = seleccionar_rol()

Seleccione su rol:
1-Registro de Actividades
2-Analista de Cargas
Opcion: 1


In [ ]:
if estado["rol"] == "registro":
  print("\n📋 REGISTRO DE ACTIVIDADES")
  print("=" * 40)

  while True:
      actividad = formulario_actividad()

      prompt = construir_prompt(actividad)
      recomendaciones = motor.generar(prompt)

      actividad["recomendaciones"] = recomendaciones
      estado["actividades"].append(actividad)

      actividades.append(actividad)

      print("\n🤖 Recomendaciones para esta actividad:\n")
      print(recomendaciones)

      continuar = input("\n¿Deseas registrar otra actividad? (s/n): ").lower()
      if continuar != "s":
          break

elif estado["rol"] == "analista":
  print("\n🟥 MODO ANALISTA DE CARGAS")
  print("=" * 40)

  if not actividades:
      print("❌ No hay actividades registradas.")
  else:
      resumen = calcular_resumen_global(actividades)

      print("\n📊 RESUMEN GLOBAL")
      print("=" * 40)
      for k, v in resumen.items():
          print(f"{k}: {v}")

      prompt_resumen = f"""
Eres un experto en análisis de cargas laborales.

Analiza el siguiente resumen y entrega:
- Diagnóstico general
- Riesgos de sobrecarga
- Tiempo muerto potencial
- Recomendación de dotación

Resumen:
{resumen}
"""
      analisis_ia = motor.generar(prompt_resumen)

      print("\n🤖 Análisis IA del resumen:\n")
      print(analisis_ia)



📋 REGISTRO DE ACTIVIDADES

📝 Registro de actividad

1️⃣ ¿Cuál es su Vicepresidencia?:
> Vp de Talento y Administrativa
1️⃣ ¿Cuál es su cargo?:
> Gerente
1️⃣ Describe la actividad que realizas:
> Liderar  el diseño y actualización del roadmap de  Delivery en Negocio
2️⃣ Unidad de medida (Correo, Reunión, Documento, etc.):
> Comite primario
3️⃣ Frecuencia (diaria / semanal / mensual / anual):
> semanal
4️⃣ Volumen según la frecuencia:
> 1
5️⃣ Tiempo real (minutos) en esta actividad:
> 60
6️⃣ Proceso del área al que pertenece:
> Consolidación del equipo de Cultura y Transformación
7️⃣ Autonomía (%) en esta actividad:
> 100
8️⃣ Etapa PHVA (Planear / Hacer / Verificar / Actuar):
> Planear
9️⃣ Observaciones (opcional):
> Sesion de alineación de prioridades con el equipo de leads de agilidad 

🤖 Recomendaciones para esta actividad:

¡Excelente actividad para analizar! Como experto en levantamiento de cargas laborales, entiendo la importancia de la eficiencia y el impacto estratégico.

Consid

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 34.243480128s.

In [ ]:
def resumen_ejecutivo(actividades):
  resumen_base = calcular_resumen_global(actividades)

  return {
      **resumen_base
  }

In [ ]:
resumen_final = resumen_ejecutivo(actividades)

print("\n📊 RESUMEN EJECUTIVO FINAL")
print("=" * 50)

for k, v in resumen_final.items():
    if k != "participacion_por_proceso":
        print(f"{k}: {v}")

#print("\n📌 PARTICIPACIÓN POR PROCESO")

#if "participacion_por_proceso" in resumen_final:
 #   for proc, datos in resumen_final["participacion_por_proceso"].items():
  #      print(f"- {proc}: {datos['porcentaje']}% ({datos['minutos_diarios']} min")
#else:
 #   print("Participación por proceso no disponible aún.")


📊 RESUMEN EJECUTIVO FINAL
carga_trabajo_phva: {'Planear': 0.0573}
total_carga_trabajo_phva: 0.0573
porcentaje_actividades_phva: {'Planear': 1.0}
cantidad_actividades_phva: {'Planear': 2}
carga_trabajo_frecuencia: {'semanal': 0.0573}
total_carga_trabajo_frecuencia: 0.0573
porcentaje_actividades_frecuencia: {'semanal': 1.0}
cantidad_actividades_frecuencia: {'semanal': 2}
carga_trabajo_proceso_area: {'Consolidación del equipo de Cultura y Transformación': 0.0573}
total_carga_trabajo_proceso_area: 0.0573
porcentaje_actividades_proceso_area: {'Consolidación del equipo de Cultura y Transformación': 1.0}
cantidad_actividades_proceso_area: {'Consolidación del equipo de Cultura y Transformación': 2}
jornada_laboral_diaria: 8.5
factor_tiempo_neto_productivo: 0.8676
horas_netas_efectivas_por_persona: 7.3746
horas_diarias_requeridas: 0.6
numero_personas: 0.081
total_carga_w_sin_tm: 0.0573
total_carga_trabajo_individual: 0.0439
minutos_diarios_empleados: 36.0
tiempo_muerto: 0.1324
porcentaje_carga

In [ ]:
prompt_dotacion = f"""
Eres un consultor experto en productividad y dotación.

Analiza este resumen de carga laboral y entrega:
- Diagnóstico claro
- Riesgos principales
- Recomendación de dotación
- Alertas de tiempo muerto o sobrecarga

Resumen:
{resumen_final}
"""

analisis_ia = motor.generar(prompt_dotacion)

print("\n🤖 ANÁLISIS IA FINAL\n")
print(analisis_ia)


🤖 ANÁLISIS IA FINAL

Como consultor experto en productividad y dotación, he analizado el resumen de carga laboral proporcionado. A continuación, presento mi diagnóstico, los riesgos principales, la recomendación de dotación y las alertas correspondientes:

---

## Análisis de Carga Laboral y Dotación: Equipo de Cultura y Transformación

### Diagnóstico Claro

El análisis de carga laboral revela una situación de **subutilización extremadamente severa** para el proceso o rol analizado, enfocado en la "Consolidación del equipo de Cultura y Transformación".

1.  **Carga Laboral Mínima Absoluta:**
    *   Las `horas_diarias_requeridas_final` son de 0.68 horas, lo que equivale a aproximadamente **41 minutos de trabajo efectivo por día**.
    *   Esta carga representa un **0.0649%** de la jornada laboral diaria teórica si se asumiera un 100% de ocupación.

2.  **Necesidad de Personal Insignificante:**
    *   El cálculo de `numero_personas` es de **0.081 FTE (equivalente a tiempo completo)**

## Exportar JSON

In [ ]:
from datetime import datetime
import json

In [ ]:
def construir_registro_historico(rol, actividades, resumen_final, analisis_ia):
  return {
      "timestamp": datetime.now().isoformat(),
      "rol": rol,
      "actividades": actividades,
      "resumen_final": resumen_final,
      "analisis_ia": analisis_ia
  }

In [ ]:
def guardar_historico(registro, nombre_archivo=None):
  if not nombre_archivo:
    fecha = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"historico_{fecha}.json"

  with open(nombre_archivo, "w", encoding="utf-8") as f:
    json.dump(registro, f, indent=2, ensure_ascii=False)

  print(f"\nHistorico guardado en: {nombre_archivo}")

In [ ]:
registro = construir_registro_historico(
    rol=estado["rol"],
    actividades=actividades,
    resumen_final=resumen_final,
    analisis_ia=analisis_ia
)

guardar_historico(registro)


Historico guardado en: historico_20260115_154146.json


In [ ]:
with open("historico_20260115_154146.json", 'r', encoding='utf-8') as f:
  data = json.load(f)

# Step 3: Pretty-print JSON with indentation
print("\n--- JSON Content ---")
print(json.dumps(data, indent=4, ensure_ascii=False))


--- JSON Content ---
{
    "timestamp": "2026-01-15T15:41:46.333358",
    "rol": "registro",
    "actividades": [
        {
            "vicepresidencia": "Vp de Talento y Administrativa",
            "cargo": "Gerente",
            "descripcion": "Liderar  el diseño y actualización del roadmap de  Delivery en Negocio",
            "unidad_medida": "Comite primario",
            "frecuencia": "semanal",
            "volumen": 1,
            "tiempo_real_min": 60,
            "proceso_area": "Desarrollo institucional de nuevas formas de trabajo y agilidad",
            "autonomia": 100,
            "phva": "Planear",
            "observaciones": "Sesion de alineación de prioridades con el equipo de leads de agilidad ",
            "recomendaciones": "¡Excelente análisis de actividad! Como experto en levantamiento de cargas laborales, valoro la precisión de los datos.\n\nAquí mi análisis resumido y conciso:\n\n**Contexto Clave:** El Gerente de la VP de Talento y Administrativa lidera un

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.tools import agent_tool
from google.adk.tools.google_search_tool import GoogleSearchTool
from google.adk.tools import url_context

mi_agente_google_search_agent = LlmAgent(
  name='Mi_agente_google_search_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in performing Google searches.'
  ),
  sub_agents=[],
  instruction='Use the GoogleSearchTool to find information on the web.',
  tools=[
    GoogleSearchTool()
  ],
)
mi_agente_url_context_agent = LlmAgent(
  name='Mi_agente_url_context_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in fetching content from URLs.'
  ),
  sub_agents=[],
  instruction='Use the UrlContextTool to retrieve content from provided URLs.',
  tools=[
    url_context
  ],
)
root_agent = LlmAgent(
  name='Mi_agente',
  model='gemini-2.5-flash',
  description=(
      'Agente para ayudar en el proceso de Levantamiento de Cargas Laborales'
  ),
  sub_agents=[],
  instruction='Quiero que actúes como un experto en levantamiento de cargas laborales, que cronometres las actividades que te vaya diciendo el usuario, saques el tiempo en el que se demora haciendo cada actividad y que después al final des una recomendación por cada actividad y sus características, las que pueden ser el cargo, unidad de medida, la descripción, la frecuencia, el volumen, el tiempo, el proceso del área, la autonomía, la etapa PHVA, Observaciones. \n\nQuiero que separes por roles al comienzo, el primer rol sería el usuario y el otro rol sería el analista. El analista necesitaría tener calculos basados en la información que se obtuvo del usuario anteriormente, como el tiempo diario, el tiempo mes, participacion actividad total, cargas sin tiempo muerto, carga de trabajo individual, carga de trabajo según el ciclo PHVA, carga de trabajo según la frecuencia, minutos diarios, horas diarias, %carga individual, %carga de trabajo, tiempo muerto, %carga de trabajo final, horas diarias final, carga segun proceso del área, horas netas efectivas, recomendación de dotación de persona ',
  tools=[
    agent_tool.AgentTool(agent=mi_agente_google_search_agent),
    agent_tool.AgentTool(agent=mi_agente_url_context_agent)
  ],
)